In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.sparse as sparse
%matplotlib inline

In [2]:
np.random.seed(12345)
s = np.random.normal(size=(3,3))
s = s@s.T

def display_cov(c):
    n = c.shape[0]
    assert c.shape[1]==n
    sd = np.sqrt(c[range(n),range(n)])
    corr = c/sd.reshape(1,-1)/sd.reshape(-1,1)
    cdecomp = pd.DataFrame(corr)
    cdecomp["sd"] = sd
    return pd.concat((cdecomp,pd.DataFrame(c)), axis=1)

display_cov(s)

,0,1,2,sd,0,1,2
0,1.000000,0.182228,-0.467632,0.735601,0.541109,0.331471,-0.283539
1,0.182228,1.000000,0.772130,2.472794,0.331471,6.114709,1.573780
2,-0.467632,0.772130,1.000000,0.824262,-0.283539,1.573780,0.679408


In [8]:
m, k, n = 1000, 20, 3
x = pd.DataFrame(np.random.normal(size=(m+k-1,n))@np.linalg.cholesky(s).T)
v = x.rolling(k).sum()

In [9]:
display_cov((x.values.T@x.values)/len(x))

,0,1,2,sd,0,1,2
0,1.000000,0.224588,-0.459865,0.745756,0.556152,0.401854,-0.271538
1,0.224588,1.000000,0.749749,2.399305,0.401854,5.756666,1.424312
2,-0.459865,0.749749,1.000000,0.791779,-0.271538,1.424312,0.626914


In [10]:
#re-estimate the covariances from the rolling sum only
A = sparse.diags([1]*k,range(k),(m,m+k-1)).toarray()
v_ = v.dropna().values
cov_est = (v_.T@np.linalg.inv(A@A.T)@v_)/m
display_cov(cov_est)

,0,1,2,sd,0,1,2
0,1.000000,0.223416,-0.456006,0.746012,0.556534,0.402198,-0.270432
1,0.223416,1.000000,0.753473,2.413127,0.402198,5.823180,1.445403
2,-0.456006,0.753473,1.000000,0.794952,-0.270432,1.445403,0.631949


In [12]:
display_cov((v_.T@v_)/m/20)

,0,1,2,sd,0,1,2
0,1.000000,0.328758,-0.366551,0.777437,0.604408,0.683250,-0.230742
1,0.328758,1.000000,0.748969,2.673245,0.683250,7.146236,1.621169
2,-0.366551,0.748969,1.000000,0.809703,-0.230742,1.621169,0.655620
